In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from dark_emulator_public import dark_emulator
import os, sys, time, json, copy
import matplotlib.pyplot as plt
from collections import OrderedDict as od
from scipy.interpolate import InterpolatedUnivariateSpline as ius
from scipy.interpolate import interp2d, interp1d
from scipy.integrate import simps
from tqdm import tqdm
import hsc3x2pt

using dark_emulator at  /Users/sugiyamasunao/Documents/python_package/dark_emulator_public/dark_emulator/__init__.py


In [3]:
power_b1 = hsc3x2pt.power_b1_class()

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [24]:
def skip_column(arr, i):
    if arr.ndim == 1:
        return np.hstack([arr[:i], arr[i+1:]])
    else:
        return np.vstack([arr[:i, :], arr[i+1:, :]])

def skipMat(mat, i):
    return skip_column(skip_column(mat, i).T, i).T

def getSigma(F, i):
    F11 = F[i,i]
    F12 = skip_column(F[i, :], i)
    F22 = skipMat(F, i)
    iF22 = np.linalg.inv(F22)

    isigma2 = F11 - np.dot(F12, np.dot(iF22, F12))
    sigma = 1.0/isigma2**0.5
    return sigma

# Year 1, single source bin analysis

In [35]:
%%time
dirname = './single_source_ClY1'
x,y,y_dict,cov, pnames, F = getFisherMat(dirname, power_b1)

CPU times: user 12.3 s, sys: 511 ms, total: 12.8 s
Wall time: 3.25 s


In [36]:
for i in range(13):
    sigma = getSigma(F, i)
    print(pnames[i], sigma)

omega_b 0.034503387229671376
omega_c 0.0731334020313097
Omega_de 0.01452649267307007
n_s 0.106338054748982
ln10p10As 0.4948429498753375
b1lowz 0.34159058659545827
b1cmass1 0.2215382070833047
b1cmass2 0.22975031866490267
alphamaglowz 65.33857389713602
alphamagcmass1 5.982145067861211
alphamagcmass2 4.349090886781821
dzph 0.42891780199605267
dm 0.408529411128198


In [6]:
pk2cl_fid = hsc3x2pt.pk2cl_class(power_b1)
pk2cl_fid.load_Cl_cache(os.path.join(dirname, 'fiducial'))

In [7]:
x = hsc3x2pt.radial_bin_class(pk2cl_fid.get_all_galaxy_sample())

In [8]:
cov = hsc3x2pt.covariance_class(x)
cov.set_covariance_from_pk2cl(pk2cl_fid)
c = cov.get_full_covariance()
cc = hsc3x2pt.correlation_coeff(c)
ic = np.linalg.inv(c)

In [9]:
y = hsc3x2pt.signal_class(x)
y.set_signal_from_pk2cl(pk2cl_fid)
v_fid = y.get_signal()

In [10]:
F = getFisherMat(dirname, power_b1, ic, v_fid)